# ETL del proyecto Steam_Recomendations

## 1. Extraccion

In [23]:
import pandas as pd
import ast

def load_json_lines(file_path):
    data = []
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            data.append(ast.literal_eval(line))
    return pd.DataFrame(data)




In [ ]:
# Cargar archivos
#df_steam_games = pd.read_json("./datasets/steam_games.json", lines=True)
#df_user_rev = load_json_lines("./datasets/user_reviews.json")
#df_user_items = load_json_lines("./datasets/users_items.json")

In [9]:
df_user_items.head(3)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


In [10]:
df_user_items['items'].loc[0]

[{'item_id': '10',
  'item_name': 'Counter-Strike',
  'playtime_forever': 6,
  'playtime_2weeks': 0},
 {'item_id': '20',
  'item_name': 'Team Fortress Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '30',
  'item_name': 'Day of Defeat',
  'playtime_forever': 7,
  'playtime_2weeks': 0},
 {'item_id': '40',
  'item_name': 'Deathmatch Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '50',
  'item_name': 'Half-Life: Opposing Force',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '60',
  'item_name': 'Ricochet',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '70',
  'item_name': 'Half-Life',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '130',
  'item_name': 'Half-Life: Blue Shift',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '300',
  'item_name': 'Day of Defeat: Source',
  'playtime_forever': 4733,
  'playtime_2weeks': 0},
 {'item_id': '240',
  'item_name': 'Counter-Strike: S

In [31]:
# Con explode se relaciona cada fila con los elementos o diccionarios en la lista 
# de diccionarios presente en la columna items
df_items_exploded = df_user_items.explode('items')

# Ahora, desanida la columna 'items' para que cada clave del diccionario se convierta en una columna
items_df = df_items_exploded['items'].apply(pd.Series)

# Concatena las columnas desanidadas de vuelta con las otras columnas de df_exploded
df_items_final = pd.concat([df_items_exploded.drop('items', axis=1), items_df], axis=1)

df_items_final.head()


,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks,0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0,NaN
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0,NaN
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0,0.0,NaN
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0.0,0.0,NaN
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0.0,0.0,NaN


In [16]:
expanded_records = []

for index,row in df_user_items.iterrows():
    user_id  = row['user_id']
    steam_id = row['steam_id']
    user_url = row['user_url']

    items_list = row['items']

    for it in items_list:
        # diccionario del nuevo df expandido
        new_item = {    'steam_id' : steam_id ,
                        'user_url' : user_url,
                        # **it
                        'item_id'  : it['item_id'],
                        'item_name'  : it['item_name'],
                        'playtime_forever'  : it['playtime_forever'],
                        'playtime_2weeks'  : it['playtime_2weeks']
                    }
        expanded_records.append(new_item)

df_exp_items = pd.DataFrame(expanded_records)

        

In [13]:
df_exp_items.head(5)

,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0,0
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7,0
3,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0,0
4,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0,0


In [29]:
df_user_rev.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [30]:
#explode() de pandas para separar cada ítem de la lista en la columna reviews en una fila separada
df_rev_exploded = df_user_rev.explode('reviews')

#cada ítem en la columna reviews debe ser un diccionario,
# la idea es desanidar ese diccionario en múltiples columnas
reviews_df = df_rev_exploded['reviews'].apply(pd.Series)

#combinar reviews_df con df_rev_exploded para obtener un solo DataFrame:
final_reviews_df = pd.concat([df_rev_exploded.drop(columns='reviews'), reviews_df], axis=1)
final_reviews_df.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,0
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,NaN
1,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,NaN


Transformacion

Carga